In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv') # this file is in files_for_lesson_and_activities folder
data#.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,Suburban,Married,69,32,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,Suburban,Single,94,13,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,Suburban,Married,108,18,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,Suburban,Married,106,18,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,Rural,Single,73,12,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,23405.987980,No,Basic,Bachelor,2/10/11,Employed,M,71941,Urban,Married,73,18,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,PK87824,California,3096.511217,Yes,Extended,College,2/12/11,Employed,F,21604,Suburban,Divorced,79,14,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,TD14365,California,8163.890428,No,Extended,Bachelor,2/6/11,Unemployed,M,0,Suburban,Single,85,9,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,UP19263,California,7524.442436,No,Extended,College,2/3/11,Employed,M,21941,Suburban,Married,96,34,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


In [3]:
data.corr()['Monthly Premium Auto'].sort_values(ascending=False)

Monthly Premium Auto             1.000000
Total Claim Amount               0.632017
Customer Lifetime Value          0.396262
Months Since Policy Inception    0.020257
Months Since Last Claim          0.005026
Number of Policies              -0.011233
Number of Open Complaints       -0.013122
Income                          -0.016665
Name: Monthly Premium Auto, dtype: float64

In [4]:
data1 = data.copy()
numericals = data1.select_dtypes(np.number).drop(columns=['Monthly Premium Auto'], axis=1) #

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# vif uses OLS to calculate the factor, so we don't have the intercept
# that's why we need to assign it:
vif_numericals = add_constant(numericals)
vif_numericals

,const,Customer Lifetime Value,Income,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount
0,1.0,2763.519279,56274,32,5,0,1,384.811147
1,1.0,6979.535903,0,13,42,0,8,1131.464935
2,1.0,12887.431650,48767,18,38,0,2,566.472247
3,1.0,7645.861827,0,18,65,0,7,529.881344
4,1.0,2813.692575,43836,12,44,0,1,138.130879
...,...,...,...,...,...,...,...,...
9129,1.0,23405.987980,71941,18,89,0,2,198.234764
9130,1.0,3096.511217,21604,14,28,0,1,379.200000
9131,1.0,8163.890428,0,9,37,3,2,790.784983
9132,1.0,7524.442436,21941,34,3,0,3,691.200000


In [6]:
# To remove multicolinearity automatically:

threshold = 30

while True:
    # calculates the vif
    values = [variance_inflation_factor(np.array(vif_numericals), i)
              for i in range(len(vif_numericals.columns))][1:]
    display(pd.DataFrame(values, index=vif_numericals.columns[1:]).sort_values(0))
    # checks if the highest vif is bigger than the threshold
    if max(values) > threshold:
        # finds the column that has the maximum vif
        col_index = values.index(max(values)) + 1
        column_name = vif_numericals.columns[col_index]
        # drops that column
        vif_numericals = vif_numericals.drop(column_name, axis=1)
    else:
        break

vif_numericals

,0
Number of Policies,1.000958
Number of Open Complaints,1.001431
Months Since Policy Inception,1.002128
Months Since Last Claim,1.002854
Customer Lifetime Value,1.070401
Income,1.160880
Total Claim Amount,1.221987


,const,Customer Lifetime Value,Income,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount
0,1.0,2763.519279,56274,32,5,0,1,384.811147
1,1.0,6979.535903,0,13,42,0,8,1131.464935
2,1.0,12887.431650,48767,18,38,0,2,566.472247
3,1.0,7645.861827,0,18,65,0,7,529.881344
4,1.0,2813.692575,43836,12,44,0,1,138.130879
...,...,...,...,...,...,...,...,...
9129,1.0,23405.987980,71941,18,89,0,2,198.234764
9130,1.0,3096.511217,21604,14,28,0,1,379.200000
9131,1.0,8163.890428,0,9,37,3,2,790.784983
9132,1.0,7524.442436,21941,34,3,0,3,691.200000


In [7]:
vif = pd.DataFrame(index=vif_numericals.columns)

vif["VIF Factor"] = [round(variance_inflation_factor(np.array(vif_numericals), i),2)
                     for i in range(len(vif_numericals.columns))]

In [8]:
vif.sort_values("VIF Factor")

,VIF Factor
Months Since Last Claim,1.00
Months Since Policy Inception,1.00
Number of Open Complaints,1.00
Number of Policies,1.00
Customer Lifetime Value,1.07
Income,1.16
Total Claim Amount,1.22
const,14.51


In [9]:
# contingency table
data_crosstab = pd.crosstab(data['Income'], data['Number of Policies'], margins=True)
data_crosstab

Number of Policies,1,2,3,4,5,6,7,8,9,All
Income,,,,,,,,,,
0,796,591,279,128,112,85,135,85,106,2317
10037,0,0,1,0,0,0,0,0,0,1
10074,1,0,0,0,0,0,0,0,0,1
10097,1,0,0,0,0,0,0,0,0,1
10105,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
99934,0,0,1,0,0,0,0,0,0,1
99960,0,0,0,1,0,0,0,0,0,1
99961,1,0,0,0,0,0,0,0,0,1


In [10]:
from scipy.stats import chi2_contingency

stats, p_value, dof, frequencies = chi2_contingency(data_crosstab)
p_value
# the result is a low p_value, so we can assume correlation between the variables

0.00020296375079291974

In [11]:
data['Income'].unique()

array([56274,     0, 48767, ..., 66367, 71941, 21941], dtype=int64)

In [12]:
income = data['Income'].apply(lambda x: 'high' if x >= 5 else 'medium' if x >= 0 and x < 5 else 'low')
gender = data['Number of Policies']

data_crosstab = pd.crosstab(income, gender, margins=True)
data_crosstab

Number of Policies,1,2,3,4,5,6,7,8,9,All
Income,,,,,,,,,,
high,2455,1703,889,281,295,287,298,299,310,6817
medium,796,591,279,128,112,85,135,85,106,2317
All,3251,2294,1168,409,407,372,433,384,416,9134


In [13]:
stats, p_value, dof, frequencies = chi2_contingency(data_crosstab)
p_value

0.2082396417225693

In [14]:
data = pd.get_dummies(data)

In [15]:
from sklearn.model_selection import train_test_split

X = data.drop('Income', axis=1)
y = data['Income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [16]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
lr.score(X_test, y_test)

0.6949647111591049

In [18]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [19]:
knn.score(X_test, y_test)

0.24071576200224365

In [20]:
from sklearn import preprocessing
import numpy as np
x_array = np.array([2,3,5,6,7,4,8,7,6])
normalized_arr = preprocessing.normalize([x_array])
print(normalized_arr)

[[0.11785113 0.1767767  0.29462783 0.35355339 0.41247896 0.23570226
  0.47140452 0.41247896 0.35355339]]


In [21]:
from sklearn.model_selection import train_test_split

X = data.drop('Income', axis=1)
y = data['Income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train_num = X_train.select_dtypes(np.number)
X_test_num = X_test.select_dtypes(np.number)

In [22]:
from sklearn.preprocessing import StandardScaler
import pickle

# Pickle - you'll use it at least for these steps:
# to save the scaler
# to save the encoder
# to save the model

transformer = StandardScaler()
transformer.fit(X_train_num)

# saving in a pickle
with open('std_transformer.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer.pickle', 'rb') as file:
    loaded_transformer = pickle.load(file)

X_train_ = loaded_transformer.transform(X_train_num)
X_test_ = loaded_transformer.transform(X_test_num)

# if you need to un-scale afterwards, speacially if you scaled target variable:
unscaled_X_train = loaded_transformer.inverse_transform(X_train_)

# it makes sense to scale the target variable as well
unscaled_X_train[0][0]

4768.955924

In [23]:
from sklearn.preprocessing import Normalizer

numericals = X.select_dtypes(np.number) # using only X, and not y

transformer = Normalizer()
transformer.fit(numericals)
x_normalized = transformer.transform(numericals)

In [24]:
from sklearn.preprocessing import MinMaxScaler

numericals = data.select_dtypes(np.number)

transformer = MinMaxScaler().fit(numericals)
x_min_max = transformer.transform(numericals)
x_min_max

array([[0.01062925, 0.56284694, 0.03375527, ..., 0.        , 1.        ,
        0.        ],
       [0.06240565, 0.        , 0.13924051, ..., 0.        , 1.        ,
        0.        ],
       [0.13495982, 0.48776267, 0.19831224, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.07695057, 0.        , 0.10126582, ..., 0.        , 1.        ,
        0.        ],
       [0.06909758, 0.2194517 , 0.14767932, ..., 1.        , 0.        ,
        0.        ],
       [0.00876645, 0.        , 0.06751055, ..., 0.        , 1.        ,
        0.        ]])

In [25]:
for col in data.select_dtypes('object'):
    print(data[col].value_counts(), '\n')

In [27]:
data["Income"] = data["Income"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})